<a href="https://colab.research.google.com/github/Kzis/mrt-is/blob/master/Prep_Model_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# อ่านข้อมูล Mrt Transaction

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# df = pd.read_csv("gdrive/My Drive/NIDA/transection_.csv" , nrows=10000)
df = pd.read_csv("gdrive/My Drive/NIDA/transection_.csv")

In [4]:
df.head()

,TransactionID,TransactionDate,CardType,BusinessDay,EntryStationID,ExitStationID,TicketType,TicketName,PassengerType,StationName,LineCode,Right_StationName,Right_LineCode,DateKey,DayType,DayNumberOfMonth,EnglishDayNameOfWeek,ThaiDayNameOfWeek,EnglishMonthName,ThaiMonthName,MonthNumberOfYear,CalendarQuater,CalendarSemester,CalendarYear
0,40,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
1,69,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
2,127,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
3,129,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
4,333,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017


# เลือกข้อมูลตามเดือนที่ต้องการ

In [5]:
def get_data_by_month(month):
  return df.loc[df['ThaiMonthName'] == month]

In [6]:
jan_df = get_data_by_month("มกราคม")
jan_df.drop('TransactionID', axis=1, inplace=True)
jan_df.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,TransactionDate,CardType,BusinessDay,EntryStationID,ExitStationID,TicketType,TicketName,PassengerType,StationName,LineCode,Right_StationName,Right_LineCode,DateKey,DayType,DayNumberOfMonth,EnglishDayNameOfWeek,ThaiDayNameOfWeek,EnglishMonthName,ThaiMonthName,MonthNumberOfYear,CalendarQuater,CalendarSemester,CalendarYear
0,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
1,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
2,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
3,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
4,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017


# Count ข้อมูลแต่ละสถานี แยกตามแต่ละวัน

In [7]:
tran_df = jan_df[['TransactionDate','StationName','CardType']].groupby(['TransactionDate','StationName']).count()
tran_df.reset_index(inplace=True)
tran_df = tran_df.rename(columns={'CardType': 'PassengerCount'})
tran_df

,TransactionDate,StationName,PassengerCount
0,2017-01-01,Bang Sue,186
1,2017-01-01,Chatuchak Park,359
2,2017-01-01,Hua Lamphong,299
3,2017-01-01,Huai Khwang,431
4,2017-01-01,Kamphaeng Phet,233
...,...,...,...
553,2017-01-31,Sam Yan,1241
554,2017-01-31,Si Lom,2384
555,2017-01-31,Sukhumvit,4891
556,2017-01-31,Sutthisan,1968


In [8]:

def get_transaction_df(month_df):
  df_month_transaction_date_sum = month_df.TransactionDate.value_counts().rename_axis('TransactionDate').reset_index(name='TransactionCounts')
  df_month_transaction_date_sum.sort_values('TransactionDate')
  return df_month_transaction_date_sum

In [9]:
transaction_df = get_transaction_df(jan_df)
transaction_df.head()

,TransactionDate,TransactionCounts
0,2017-01-25,37135
1,2017-01-26,36485
2,2017-01-13,36171
3,2017-01-24,36050
4,2017-01-20,35898


# Preparation ข้อมูลประเภทของวัน (วันทำงาน , วันหยุด , วันนักขัตฤกษ์)

In [10]:
def get_day_type_list(month_df , month_transaction_df , col_name):
    date_list = month_transaction_df['TransactionDate'].tolist()
    col_list = month_df[col_name].unique().tolist()
    print(col_list)
    output = []
    for date in date_list:
      lst = []
      lst.append(date)
      for col_val in col_list:
        subset_df = month_df[ (month_df[col_name]==col_val) & (month_df["TransactionDate"]==date) ].count()
        column_count = subset_df.value_counts()
        lst.append(column_count.index[0])
        # print("{} : {} : {}".format(date,col_val,column_count.index[0]) )
      # print("-----")
      output.append(lst)
    print(output)
    return output
    # df = pd.DataFrame(output, columns =['TransactionDate','Type', 'Station','Count']) 
    # return df

In [11]:
day_type_list = get_day_type_list(jan_df , transaction_df , "DayType")

['WE', 'PH', 'WD']
[['2017-01-25', 0, 0, 37135], ['2017-01-26', 0, 0, 36485], ['2017-01-13', 0, 0, 36171], ['2017-01-24', 0, 0, 36050], ['2017-01-20', 0, 0, 35898], ['2017-01-11', 0, 0, 35831], ['2017-01-18', 0, 0, 35575], ['2017-01-10', 0, 0, 35343], ['2017-01-31', 0, 0, 35253], ['2017-01-19', 0, 0, 35208], ['2017-01-17', 0, 0, 35107], ['2017-01-09', 0, 0, 34666], ['2017-01-23', 0, 0, 34660], ['2017-01-06', 0, 0, 33946], ['2017-01-27', 0, 0, 33717], ['2017-01-16', 0, 0, 33200], ['2017-01-05', 0, 0, 33129], ['2017-01-30', 0, 0, 32814], ['2017-01-12', 0, 0, 31840], ['2017-01-04', 0, 0, 31479], ['2017-01-28', 16397, 0, 0], ['2017-01-21', 16184, 0, 0], ['2017-01-14', 16018, 0, 0], ['2017-01-07', 15156, 0, 0], ['2017-01-29', 13026, 0, 0], ['2017-01-03', 0, 12134, 0], ['2017-01-22', 12071, 0, 0], ['2017-01-15', 12068, 0, 0], ['2017-01-08', 11123, 0, 0], ['2017-01-02', 0, 7217, 0], ['2017-01-01', 5045, 0, 0]]


In [12]:
day_type_temp = pd.DataFrame(day_type_list, columns =['Date','WE','PH','WD'])
day_type_temp['DayType'] = ( (day_type_temp["WE"] != 0) * 1 ) + ( (day_type_temp["PH"] != 0) * 2 ) + ( (day_type_temp["WD"] != 0) * 3 )
day_type_temp.loc[day_type_temp['DayType'] == 1, 'DayType'] = "WE"
day_type_temp.loc[day_type_temp['DayType'] == 2, 'DayType'] = "PH"
day_type_temp.loc[day_type_temp['DayType'] == 3, 'DayType'] = "WD"
day_type_temp.drop('WE', axis=1, inplace=True)
day_type_temp.drop('PH', axis=1, inplace=True)
day_type_temp.drop('WD', axis=1, inplace=True)

In [13]:
day_type_temp.head()

,Date,DayType
0,2017-01-25,WD
1,2017-01-26,WD
2,2017-01-13,WD
3,2017-01-24,WD
4,2017-01-20,WD


In [14]:
jan_transaction_df = pd.merge(left=tran_df, right=day_type_temp, left_on='TransactionDate', right_on='Date')
jan_transaction_df.drop('Date', axis=1, inplace=True)
jan_transaction_df

,TransactionDate,StationName,PassengerCount,DayType
0,2017-01-01,Bang Sue,186,WE
1,2017-01-01,Chatuchak Park,359,WE
2,2017-01-01,Hua Lamphong,299,WE
3,2017-01-01,Huai Khwang,431,WE
4,2017-01-01,Kamphaeng Phet,233,WE
...,...,...,...,...
553,2017-01-31,Sam Yan,1241,WD
554,2017-01-31,Si Lom,2384,WD
555,2017-01-31,Sukhumvit,4891,WD
556,2017-01-31,Sutthisan,1968,WD


# Preparation ข้อมูลวันอะไรของสัปดาห์

In [15]:
def get_day_of_week_list(month_df , month_transaction_df , col_name):
    date_list = month_transaction_df['TransactionDate'].tolist()
    col_list = month_df[col_name].unique().tolist()
    print(col_list)
    output = []
    for date in date_list:
      lst = []
      lst.append(date)
      for col_val in col_list:
        subset_df = month_df[ (month_df[col_name]==col_val) & (month_df["TransactionDate"]==date) ].count()
        column_count = subset_df.value_counts()
        lst.append(column_count.index[0])
        # print("{} : {} : {}".format(date,col_val,column_count.index[0]) )
      # print("-----")
      output.append(lst)
    print(output)
    return output

In [16]:
day_of_week_list = get_day_of_week_list(jan_df , transaction_df , "EnglishDayNameOfWeek")

['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
[['2017-01-25', 0, 0, 0, 37135, 0, 0, 0], ['2017-01-26', 0, 0, 0, 0, 36485, 0, 0], ['2017-01-13', 0, 0, 0, 0, 0, 36171, 0], ['2017-01-24', 0, 0, 36050, 0, 0, 0, 0], ['2017-01-20', 0, 0, 0, 0, 0, 35898, 0], ['2017-01-11', 0, 0, 0, 35831, 0, 0, 0], ['2017-01-18', 0, 0, 0, 35575, 0, 0, 0], ['2017-01-10', 0, 0, 35343, 0, 0, 0, 0], ['2017-01-31', 0, 0, 35253, 0, 0, 0, 0], ['2017-01-19', 0, 0, 0, 0, 35208, 0, 0], ['2017-01-17', 0, 0, 35107, 0, 0, 0, 0], ['2017-01-09', 0, 34666, 0, 0, 0, 0, 0], ['2017-01-23', 0, 34660, 0, 0, 0, 0, 0], ['2017-01-06', 0, 0, 0, 0, 0, 33946, 0], ['2017-01-27', 0, 0, 0, 0, 0, 33717, 0], ['2017-01-16', 0, 33200, 0, 0, 0, 0, 0], ['2017-01-05', 0, 0, 0, 0, 33129, 0, 0], ['2017-01-30', 0, 32814, 0, 0, 0, 0, 0], ['2017-01-12', 0, 0, 0, 0, 31840, 0, 0], ['2017-01-04', 0, 0, 0, 31479, 0, 0, 0], ['2017-01-28', 0, 0, 0, 0, 0, 0, 16397], ['2017-01-21', 0, 0, 0, 0, 0, 0, 16184], ['2017-01-14', 0, 

In [17]:
day_of_week_unique = jan_df['EnglishDayNameOfWeek'].unique().tolist()
day_of_week_unique.insert(0,"Date")

In [18]:
day_of_week_temp = pd.DataFrame(day_of_week_list, columns =day_of_week_unique)
day_of_week_temp['DateOfWeek'] = ( (day_of_week_temp["Sunday"] != 0) * 1 ) + ( (day_of_week_temp["Monday"] != 0) * 2 ) + ( (day_of_week_temp["Tuesday"] != 0) * 3 )+ ( (day_of_week_temp["Wednesday"] != 0) * 4 )+ ( (day_of_week_temp["Thursday"] != 0) * 5 )+ ( (day_of_week_temp["Friday"] != 0) * 6 )+ ( (day_of_week_temp["Saturday"] != 0) * 7 )
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 1, 'DateOfWeek'] = "Sunday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 2, 'DateOfWeek'] = "Monday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 3, 'DateOfWeek'] = "Tuesday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 4, 'DateOfWeek'] = "Wednesday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 5, 'DateOfWeek'] = "Thursday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 6, 'DateOfWeek'] = "Friday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 7, 'DateOfWeek'] = "Saturday"
for col in jan_df['EnglishDayNameOfWeek'].unique().tolist():
  day_of_week_temp.drop(col, axis=1, inplace=True)

In [19]:
jan_transaction_df = pd.merge(left=jan_transaction_df, right=day_of_week_temp, left_on='TransactionDate', right_on='Date')
jan_transaction_df.drop('Date', axis=1, inplace=True)
jan_transaction_df

,TransactionDate,StationName,PassengerCount,DayType,DateOfWeek
0,2017-01-01,Bang Sue,186,WE,Sunday
1,2017-01-01,Chatuchak Park,359,WE,Sunday
2,2017-01-01,Hua Lamphong,299,WE,Sunday
3,2017-01-01,Huai Khwang,431,WE,Sunday
4,2017-01-01,Kamphaeng Phet,233,WE,Sunday
...,...,...,...,...,...
553,2017-01-31,Sam Yan,1241,WD,Tuesday
554,2017-01-31,Si Lom,2384,WD,Tuesday
555,2017-01-31,Sukhumvit,4891,WD,Tuesday
556,2017-01-31,Sutthisan,1968,WD,Tuesday


# Preparation ข้อมูล วันที่เท่าไหร่ของเดือน และ เป็นช่วงไหนของเดือน (ต้นเดือน , กลางเดือน , สิ้นเดือน)

In [20]:
df_day_number_of_month = pd.DataFrame(columns=['DayNumberOfMonth','MonthPeriod','MonthNumberOfYear','CalendarQuater'])
df_day_number_of_month["DayNumberOfMonth"] =  jan_transaction_df['TransactionDate'].str.slice(8, 10).astype('int')
df_day_number_of_month["MonthPeriod"] =  jan_transaction_df['TransactionDate'].str.slice(8, 10).astype('int')
df_day_number_of_month["MonthNumberOfYear"] =  jan_transaction_df['TransactionDate'].str.slice(5, 7).astype('int')

In [21]:
df_day_number_of_month.loc[df_day_number_of_month['MonthPeriod'].astype('int') <= 5, 'MonthPeriod'] = 1
df_day_number_of_month.loc[( (df_day_number_of_month['MonthPeriod'].astype('int') > 5) & (df_day_number_of_month['MonthPeriod'].astype('int') < 25) ), 'MonthPeriod'] = 2
df_day_number_of_month.loc[df_day_number_of_month['MonthPeriod'].astype('int') >= 25, 'MonthPeriod'] = 3

df_day_number_of_month.loc[df_day_number_of_month['MonthPeriod'] == 1, 'MonthPeriod'] = "Early"
df_day_number_of_month.loc[df_day_number_of_month['MonthPeriod'] == 2, 'MonthPeriod'] = "Mid"
df_day_number_of_month.loc[df_day_number_of_month['MonthPeriod'] == 3, 'MonthPeriod'] = "Late"

In [22]:
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 1, 'CalendarQuater'] = 1
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 2, 'CalendarQuater'] = 1
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 3, 'CalendarQuater'] = 1
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 4, 'CalendarQuater'] = 2
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 5, 'CalendarQuater'] = 2
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 6, 'CalendarQuater'] = 2
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 7, 'CalendarQuater'] = 3
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 8, 'CalendarQuater'] = 3
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 9, 'CalendarQuater'] = 3
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 10, 'CalendarQuater'] = 4
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 11, 'CalendarQuater'] = 4
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 12, 'CalendarQuater'] = 4

In [23]:
jan_transaction_df = jan_transaction_df.join(df_day_number_of_month)
jan_transaction_df

,TransactionDate,StationName,PassengerCount,DayType,DateOfWeek,DayNumberOfMonth,MonthPeriod,MonthNumberOfYear,CalendarQuater
0,2017-01-01,Bang Sue,186,WE,Sunday,1,Early,1,1
1,2017-01-01,Chatuchak Park,359,WE,Sunday,1,Early,1,1
2,2017-01-01,Hua Lamphong,299,WE,Sunday,1,Early,1,1
3,2017-01-01,Huai Khwang,431,WE,Sunday,1,Early,1,1
4,2017-01-01,Kamphaeng Phet,233,WE,Sunday,1,Early,1,1
...,...,...,...,...,...,...,...,...,...
553,2017-01-31,Sam Yan,1241,WD,Tuesday,31,Late,1,1
554,2017-01-31,Si Lom,2384,WD,Tuesday,31,Late,1,1
555,2017-01-31,Sukhumvit,4891,WD,Tuesday,31,Late,1,1
556,2017-01-31,Sutthisan,1968,WD,Tuesday,31,Late,1,1


# อ่านข้อมูล Events

In [24]:
events_df = pd.read_csv("gdrive/My Drive/NIDA/Events.csv")
events_df.drop('EventsId', axis=1, inplace=True)

In [25]:
events_df.head()

,EvenstName,EventsLocation,EventsType,Interest,MRTStation,EventsDate
0,LEGAL EXECUTION DEPARTMENT AUCTION 2017 Ep.1,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),กฏหมาย,2241,Queen Sirikit National Convention Centre,2017-01-08
1,19th Bangkok International Symposium on HIV Me...,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),สัมมนา,1565,Queen Sirikit National Convention Centre,2017-01-19
2,19th Bangkok International Symposium on HIV Me...,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),สัมมนา,1565,Queen Sirikit National Convention Centre,2017-01-20
3,IDP AUSportunity,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),การศึกษา,1229,Queen Sirikit National Convention Centre,2017-01-21
4,IDP AUSportunity,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),การศึกษา,1229,Queen Sirikit National Convention Centre,2017-01-22


In [26]:
events_df.shape

(920, 6)

In [27]:
events_df.MRTStation.unique()

array(['Queen Sirikit National Convention Centre', 'Phahon Yothin',
       'Phra Ram 9', 'Sukhumvit'], dtype=object)

In [28]:
jan_transaction_df.StationName.unique()

array(['Bang Sue', 'Chatuchak Park', 'Hua Lamphong', 'Huai Khwang',
       'Kamphaeng Phet', 'Khlong Toei', 'Lat Phrao', 'Lumphini',
       'Phahon Yothin', 'Phetchaburi', 'Phra Ram 9',
       'Queen Sirikit National Convention Centre', 'Ratchadaphisek',
       'Sam Yan', 'Si Lom', 'Sukhumvit', 'Sutthisan',
       'Thailand Cultural Centre'], dtype=object)

In [29]:
events_df.loc[events_df.EventsDate == "2/29/2017" , "EventsDate"] = "2017-02-29"

In [30]:
events_df["EventsGroup"] = np.nan
events_df.loc[events_df.EventsType.isin(["ทั่วไป"]) , "EventsGroup"] = "General"
events_df.loc[events_df.EventsType.isin(["แฟชั่น","ความบันเทิง","Design"]) , "EventsGroup"] = "Fasion"
events_df.loc[events_df.EventsType.isin(["ที่อยู่อาศัย"]) , "EventsGroup"] = "Residence"
events_df.loc[events_df.EventsType.isin(["อาหาร"]) , "EventsGroup"] = "Foods"
events_df.loc[events_df.EventsType.isin(["กีฬา"]) , "EventsGroup"] = "Sports"
events_df.loc[events_df.EventsType.isin(["สัมมนา"]) , "EventsGroup"] = "Seminar"
events_df.loc[events_df.EventsType.isin(["สินค้า IT","เครื่องใช้ไฟฟ้า"]) , "EventsGroup"] = "IT"
events_df.loc[events_df.EventsType.isin(["งานแต่ง","ครอบครัว"]) , "EventsGroup"] = "Family"
events_df.loc[events_df.EventsType.isin(["สุขภาพ","Beauty"]) , "EventsGroup"] = "Health"
events_df.loc[events_df.EventsType.isin(["การศึกษา","กฏหมาย"]) , "EventsGroup"] = "Education"
events_df.loc[events_df.EventsType.isin(["ธุรกิจ","อาชีพ","การเงิน"]) , "EventsGroup"] = "Business"
events_df.loc[events_df.EventsType.isin(["สัตว์เลี้ยง","เกษตร"]) , "EventsGroup"] = "Agriculture"
events_df.loc[events_df.EventsType.isin(["หนังสือ"]) , "EventsGroup"] = "Books"
events_df.loc[events_df.EventsType.isin(["ท่องเที่ยว"]) , "EventsGroup"] = "Travel"
events_df.loc[events_df.EventsType.isin(["พาหนะ"]) , "EventsGroup"] = "Vehnicle"
events_df.EventsGroup.value_counts()

General        198
Fasion         171
Residence      100
Foods           71
Sports          67
IT              60
Business        50
Seminar         43
Travel          37
Agriculture     35
Books           28
Family          20
Education       14
Health          13
Vehnicle        13
Name: EventsGroup, dtype: int64

In [31]:
events_df

,EvenstName,EventsLocation,EventsType,Interest,MRTStation,EventsDate,EventsGroup
0,LEGAL EXECUTION DEPARTMENT AUCTION 2017 Ep.1,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),กฏหมาย,2241,Queen Sirikit National Convention Centre,2017-01-08,Education
1,19th Bangkok International Symposium on HIV Me...,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),สัมมนา,1565,Queen Sirikit National Convention Centre,2017-01-19,Seminar
2,19th Bangkok International Symposium on HIV Me...,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),สัมมนา,1565,Queen Sirikit National Convention Centre,2017-01-20,Seminar
3,IDP AUSportunity,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),การศึกษา,1229,Queen Sirikit National Convention Centre,2017-01-21,Education
4,IDP AUSportunity,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),การศึกษา,1229,Queen Sirikit National Convention Centre,2017-01-22,Education
...,...,...,...,...,...,...,...
915,Shine Flea market ตอน Thai Street Food,TERMINAL 21,ทั่วไป,3542,Sukhumvit,2017-09-23,General
916,Shine Flea market ตอน Thai Street Food,TERMINAL 21,ทั่วไป,3542,Sukhumvit,2017-09-24,General
917,Gift Fest 2018,TERMINAL 21,ทั่วไป,1437,Sukhumvit,2017-12-22,General
918,Gift Fest 2018,TERMINAL 21,ทั่วไป,1437,Sukhumvit,2017-12-23,General


In [32]:
events_group_df = pd.get_dummies(data=events_df , columns=["EventsGroup"])
events_group_df.head()

,EvenstName,EventsLocation,EventsType,Interest,MRTStation,EventsDate,EventsGroup_Agriculture,EventsGroup_Books,EventsGroup_Business,EventsGroup_Education,EventsGroup_Family,EventsGroup_Fasion,EventsGroup_Foods,EventsGroup_General,EventsGroup_Health,EventsGroup_IT,EventsGroup_Residence,EventsGroup_Seminar,EventsGroup_Sports,EventsGroup_Travel,EventsGroup_Vehnicle
0,LEGAL EXECUTION DEPARTMENT AUCTION 2017 Ep.1,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),กฏหมาย,2241,Queen Sirikit National Convention Centre,2017-01-08,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,19th Bangkok International Symposium on HIV Me...,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),สัมมนา,1565,Queen Sirikit National Convention Centre,2017-01-19,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,19th Bangkok International Symposium on HIV Me...,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),สัมมนา,1565,Queen Sirikit National Convention Centre,2017-01-20,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,IDP AUSportunity,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),การศึกษา,1229,Queen Sirikit National Convention Centre,2017-01-21,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,IDP AUSportunity,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),การศึกษา,1229,Queen Sirikit National Convention Centre,2017-01-22,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [33]:
events_group_df.shape

(920, 21)

In [34]:
events_group_name = ["General","Fasion","Residence","Foods","Sports","Seminar","IT","Family","Health","Education","Business","Agriculture","Books","Travel","Vehnicle"]

In [43]:
interest_df = events_df.groupby(["EventsDate","MRTStation","EventsGroup"]).sum()
interest_df

Interest
EventsDate MRTStation    EventsGroup          
2017-01-05 Phahon Yothin Foods            5294
                         General          1553
2017-01-06 Phahon Yothin Foods            5294
                         General          1553
2017-01-07 Phahon Yothin Foods            5294
...                                        ...
2017-12-30 Phahon Yothin General          7341
2017-12-31 Phahon Yothin General          7341
2561-01-01 Phahon Yothin General          7341
2561-01-02 Phahon Yothin General          7341
2561-01-03 Phahon Yothin General          7341

[807 rows x 1 columns]

In [49]:
events_interest_df = pd.pivot_table(data=interest_df , values="Interest" , columns="EventsGroup", index=["EventsDate","MRTStation"])
events_interest_df.reset_index(inplace=True)

In [50]:
events_interest_df

EventsGroup,EventsDate,MRTStation,Agriculture,Books,Business,Education,Family,Fasion,Foods,General,Health,IT,Residence,Seminar,Sports,Travel,Vehnicle
0,2017-01-05,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,5294.0,1553.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-06,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,5294.0,1553.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-07,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,5294.0,1553.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-08,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,5294.0,1553.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-08,Queen Sirikit National Convention Centre,NaN,NaN,NaN,2241.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,2017-12-30,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7341.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
599,2017-12-31,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7341.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600,2561-01-01,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7341.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
601,2561-01-02,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7341.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
events_interest_df["EventsGroup_Agriculture"] = np.nan
events_interest_df.loc[events_interest_df.Agriculture > 0 , "EventsGroup_Agriculture"] = 1

events_interest_df["EventsGroup_Bookss"] = np.nan
events_interest_df.loc[events_interest_df.Books > 0 , "EventsGroup_Books"] = 1

events_interest_df["EventsGroup_Business"] = np.nan
events_interest_df.loc[events_interest_df.Business > 0 , "EventsGroup_Business"] = 1

events_interest_df["EventsGroup_Education"] = np.nan
events_interest_df.loc[events_interest_df.Education > 0 , "EventsGroup_Education"] = 1

events_interest_df["EventsGroup_Family"] = np.nan
events_interest_df.loc[events_interest_df.Family > 0 , "EventsGroup_Family"] = 1

events_interest_df["EventsGroup_Fasion"] = np.nan
events_interest_df.loc[events_interest_df.Fasion > 0 , "EventsGroup_Fasion"] = 1

events_interest_df["EventsGroup_Foods"] = np.nan
events_interest_df.loc[events_interest_df.Foods > 0 , "EventsGroup_Foods"] = 1

events_interest_df["EventsGroup_General"] = np.nan
events_interest_df.loc[events_interest_df.General > 0 , "EventsGroup_General"] = 1

events_interest_df["EventsGroup_Health"] = np.nan
events_interest_df.loc[events_interest_df.Health > 0 , "EventsGroup_Health"] = 1

events_interest_df["EventsGroup_IT"] = np.nan
events_interest_df.loc[events_interest_df.IT > 0 , "EventsGroup_IT"] = 1

events_interest_df["EventsGroup_Residence"] = np.nan
events_interest_df.loc[events_interest_df.Residence > 0 , "EventsGroup_Residence"] = 1

events_interest_df["EventsGroup_Seminar"] = np.nan
events_interest_df.loc[events_interest_df.Seminar > 0 , "EventsGroup_Seminar"] = 1

events_interest_df["EventsGroup_Sports"] = np.nan
events_interest_df.loc[events_interest_df.Sports > 0 , "EventsGroup_Sports"] = 1

events_interest_df["EventsGroup_Travel"] = np.nan
events_interest_df.loc[events_interest_df.Travel > 0 , "EventsGroup_Travel"] = 1

events_interest_df["EventsGroup_Vehnicle"] = np.nan
events_interest_df.loc[events_interest_df.Vehnicle > 0 , "EventsGroup_Vehnicle"] = 1

In [52]:
events_interest_df

EventsGroup,EventsDate,MRTStation,Agriculture,Books,Business,Education,Family,Fasion,Foods,General,Health,IT,Residence,Seminar,Sports,Travel,Vehnicle,EventsGroup_Agriculture,EventsGroup_Bookss,EventsGroup_Books,EventsGroup_Business,EventsGroup_Education,EventsGroup_Family,EventsGroup_Fasion,EventsGroup_Foods,EventsGroup_General,EventsGroup_Health,EventsGroup_IT,EventsGroup_Residence,EventsGroup_Seminar,EventsGroup_Sports,EventsGroup_Travel,EventsGroup_Vehnicle
0,2017-01-05,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,5294.0,1553.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-06,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,5294.0,1553.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-07,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,5294.0,1553.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-08,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,5294.0,1553.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-08,Queen Sirikit National Convention Centre,NaN,NaN,NaN,2241.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,2017-12-30,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7341.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
599,2017-12-31,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7341.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600,2561-01-01,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7341.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
601,2561-01-02,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7341.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Preparation ข้อมูล Events

In [39]:
xjan_transaction_df = pd.merge(left=jan_transaction_df,how='left', right=events_df, left_on=['StationName','TransactionDate'], right_on=['MRTStation','EventsDate'])
xjan_transaction_df

,TransactionDate,StationName,PassengerCount,DayType,DateOfWeek,DayNumberOfMonth,MonthPeriod,MonthNumberOfYear,CalendarQuater,EvenstName,EventsLocation,EventsType,Interest,MRTStation,EventsDate,EventsGroup
0,2017-01-01,Bang Sue,186,WE,Sunday,1,Early,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-01,Chatuchak Park,359,WE,Sunday,1,Early,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-01,Hua Lamphong,299,WE,Sunday,1,Early,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-01,Huai Khwang,431,WE,Sunday,1,Early,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-01,Kamphaeng Phet,233,WE,Sunday,1,Early,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,2017-01-31,Sam Yan,1241,WD,Tuesday,31,Late,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
584,2017-01-31,Si Lom,2384,WD,Tuesday,31,Late,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
585,2017-01-31,Sukhumvit,4891,WD,Tuesday,31,Late,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
586,2017-01-31,Sutthisan,1968,WD,Tuesday,31,Late,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
xjan_transaction_df.to_excel (r'Jan_Summary.xlsx', index = False, header=True)